# Dependencies

In [ ]:
from datetime import date
from typing import Optional, List
from sqlalchemy import create_engine, text, Integer, String, ForeignKey, Date
from sqlalchemy.engine import URL
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.exc import ProgrammingError

# Establish a connection

In [ ]:
server_name = "Alstation"
username = "sa"
password = "D@tabases"

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};UID={username};PWD={password}"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(url_string)

# Create a database

In [ ]:
database_name = "bookstore"

try:
    with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(text(f"CREATE DATABASE {database_name}"))
except ProgrammingError as err:
    err_string = str(err)
    if "'bookstore' already exists" in err_string:
        print(f"A database named '{database_name}' already exists.")
    else:
        print(err)

A database named 'bookstore' already exists.


# Create tables

## Required

In [ ]:
class Base(DeclarativeBase):
    pass

class Authors(Base):
    __tablename__ = "authors"

    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    birth_date: Mapped[Optional[date]] = mapped_column(Date)

    # deleting an author deletes all their books
    books: Mapped[List["Books"]] = relationship(back_populates="authors", cascade="all, delete-orphan")

class Books(Base):
    __tablename__ = "books"

    isbn: Mapped[int] = mapped_column(primary_key=True)
    title: Mapped[str] = mapped_column(String(100))
    language: Mapped[Optional[str]] = mapped_column(String(30))
    price: Mapped[int] = mapped_column(Integer)
    publication_date: Mapped[Optional[date]] = mapped_column(Date)
    author_id: Mapped[int] = mapped_column(ForeignKey("authors.id"))

    # deleting a book still keeps the inventory for that book
    inventory: Mapped[List["Inventory"]] = relationship(back_populates="books", cascade="all")

class Stores(Base):
    __tablename__ = "stores"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(50))
    address: Mapped[Optional[str]] = mapped_column(String(100))

    # deleting a store deletes it's inventory
    inventory: Mapped[List["Inventory"]] = relationship(back_populates="stores", cascade="all, delete-orphan")

class Inventory(Base):
    __tablename__ = "inventory"

    store_id: Mapped[int] = mapped_column(ForeignKey("stores.id"))
    isbn: Mapped[int] = mapped_column(ForeignKey("books.isbn"))
    amount: Mapped[int] = mapped_column(Integer)

## Additional